In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tfrecorder import TFrecorder
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%pylab inline

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Populating the interactive namespace from numpy and matplotlib


In [2]:
tfr = TFrecorder()
def input_fn_maker(path, data_info_path, shuffle=False, batch_size = 1, epoch = 1, padding = None):
    def input_fn():
        filenames = tfr.get_filenames(path=path, shuffle=shuffle)
        dataset=tfr.get_dataset(paths=filenames, data_info=data_info_path, shuffle = shuffle, 
                            batch_size = batch_size, epoch = epoch, padding =padding)
        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()
    return input_fn

padding_info = ({'image':[784,],'label':[]})
test_input_fn = input_fn_maker('mnist_tfrecord/test/',  'mnist_tfrecord/data_info.csv',batch_size = 512,
                               padding = padding_info)
train_input_fn = input_fn_maker('mnist_tfrecord/train/',  'mnist_tfrecord/data_info.csv', shuffle=True, batch_size = 128,
                               padding = padding_info)
train_eval_fn = input_fn_maker('mnist_tfrecord/train/',  'mnist_tfrecord/data_info.csv', batch_size = 512,
                               padding = padding_info)

In [3]:
def model_fn(features, mode):
    # reshape 784维的图片到28x28的平面表达，1为channel数
    features['image'] = tf.reshape(features['image'],[-1,28,28,1])
    # shape: [None,28,28,1]
    # create RNN cells:
    rnn_cells = [tf.nn.rnn_cell.GRUCell(dim) for dim in [128,256]]
    # stack cells for multi-layers RNN
    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_cells)
    # create RNN layers
    outputs, last_state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                                   inputs=tf.reshape(features['image'],[-1,28,28]),
                                   dtype=tf.float32)
    # shape: outputs: [None,28,256]
    # shape: last_state: [None,256]
    dense1 = tf.layers.dense(inputs=last_state[1], units=1024, activation=tf.nn.relu, name= 'dense1')
    # shape: [None,1024]
    dropout = tf.layers.dropout(inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dense1, units=10, name= 'output')
    # shape: [None,10]
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "labels": features['label'],
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
        }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=features['label'], logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=features['label'], predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
rm -rf mnist_model_rnn/

In [5]:
mnist_classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir="mnist_model_rnn")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'mnist_model_rnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f52e0f70ac8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
mnist_classifier.train(
    input_fn=train_input_fn)

read dataframe from mnist_tfrecord/train/train18000_19000.tfrecord x 55
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into mnist_model_rnn/model.ckpt.
INFO:tensorflow:loss = 2.30266, step = 1
INFO:tensorflow:global_step/sec: 8.19347
INFO:tensorflow:loss = 0.332536, step = 101 (12.207 sec)
INFO:tensorflow:global_step/sec: 8.40164
INFO:tensorflow:loss = 0.121271, step = 201 (11.903 sec)
INFO:tensorflow:global_step/sec: 8.34842
INFO:tensorflow:loss = 0.1062, step = 301 (11.978 sec)
INFO:tensorflow:global_step/sec: 8.34346
INFO:tensorflow:loss = 0.211641, step = 401 (11.986 sec)
INFO:tensorflow:Saving checkpoints for 430 into mnist_model_rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.232717.


In [7]:
eval_results = mnist_classifier.evaluate(input_fn=train_eval_fn)
print('train set')
print(eval_results)

read dataframe from mnist_tfrecord/train/train19000_20000.tfrecord x 55
INFO:tensorflow:Starting evaluation at 2018-04-07-14:02:27
INFO:tensorflow:Restoring parameters from mnist_model_rnn/model.ckpt-430
INFO:tensorflow:Finished evaluation at 2018-04-07-14:02:39
INFO:tensorflow:Saving dict for global step 430: accuracy = 0.959727, global_step = 430, loss = 0.127482
train set
{'accuracy': 0.95972729, 'loss': 0.12748231, 'global_step': 430}


In [8]:
eval_results = mnist_classifier.evaluate(input_fn=test_input_fn)
print('test set')
print(eval_results)

read dataframe from mnist_tfrecord/test/test1000_2000.tfrecord x 10
INFO:tensorflow:Starting evaluation at 2018-04-07-14:02:41
INFO:tensorflow:Restoring parameters from mnist_model_rnn/model.ckpt-430
INFO:tensorflow:Finished evaluation at 2018-04-07-14:02:44
INFO:tensorflow:Saving dict for global step 430: accuracy = 0.9615, global_step = 430, loss = 0.122945
test set
{'accuracy': 0.96149999, 'loss': 0.12294455, 'global_step': 430}


In [9]:
# train 2 more times
for i in range(2):
    mnist_classifier.train(
        input_fn=train_input_fn)

read dataframe from mnist_tfrecord/train/train3000_4000.tfrecord x 55
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from mnist_model_rnn/model.ckpt-430
INFO:tensorflow:Saving checkpoints for 431 into mnist_model_rnn/model.ckpt.
INFO:tensorflow:loss = 0.0984474, step = 431
INFO:tensorflow:global_step/sec: 8.30483
INFO:tensorflow:loss = 0.160819, step = 531 (12.043 sec)
INFO:tensorflow:global_step/sec: 8.70114
INFO:tensorflow:loss = 0.0891726, step = 631 (11.493 sec)
INFO:tensorflow:global_step/sec: 8.63201
INFO:tensorflow:loss = 0.0167433, step = 731 (11.585 sec)
INFO:tensorflow:global_step/sec: 8.6374
INFO:tensorflow:loss = 0.0962518, step = 831 (11.577 sec)
INFO:tensorflow:Saving checkpoints for 860 into mnist_model_rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0843372.
read dataframe from mnist_tfrecord/train/train22000_23000.tfrecord x 55
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from mnist_model_rnn/

In [10]:
eval_results = mnist_classifier.evaluate(input_fn=train_eval_fn)
print('train set')
print(eval_results)

read dataframe from mnist_tfrecord/train/train19000_20000.tfrecord x 55
INFO:tensorflow:Starting evaluation at 2018-04-07-14:05:31
INFO:tensorflow:Restoring parameters from mnist_model_rnn/model.ckpt-1290
INFO:tensorflow:Finished evaluation at 2018-04-07-14:05:43
INFO:tensorflow:Saving dict for global step 1290: accuracy = 0.987, global_step = 1290, loss = 0.0402703
train set
{'accuracy': 0.98699999, 'loss': 0.04027031, 'global_step': 1290}


In [11]:
eval_results = mnist_classifier.evaluate(input_fn=test_input_fn)
print('test set')
print(eval_results)

read dataframe from mnist_tfrecord/test/test1000_2000.tfrecord x 10
INFO:tensorflow:Starting evaluation at 2018-04-07-14:05:45
INFO:tensorflow:Restoring parameters from mnist_model_rnn/model.ckpt-1290
INFO:tensorflow:Finished evaluation at 2018-04-07-14:05:48
INFO:tensorflow:Saving dict for global step 1290: accuracy = 0.9848, global_step = 1290, loss = 0.052044
test set
{'accuracy': 0.98479998, 'loss': 0.052043963, 'global_step': 1290}
